# claudette-pydantic

> 

> Adds Pydantic support for [claudette](https://github.com/AnswerDotAI/claudette) through function calling


claudette_pydantic provides the `struct` method in the `Client` and `Chat` of claudette

`struct` provides a wrapper around `__call__`. Provide a Pydantic `BaseModel` as schema, and the model will return an initialized `BaseModel` object.

I've found Haiku to be quite reliable at even complicated schemas.

## Install

```sh
pip install claudette_pydantic
```

## Getting Started

In [45]:
from claudette.core import *
import claudette_pydantic
from pydantic import BaseModel, Field
from typing import Literal, Union, List

In [46]:
model = models[-1]
model

'claude-3-haiku-20240307'

### Simple Model

In [47]:
class Pet(BaseModel):
    "Create a new pet"
    name: str
    age: int
    owner: str = Field(default="NA", description="Owner name. Do not return if not given.")
    type: Literal['dog', 'cat', 'mouse']

c = Client(model)
print(repr(c.struct(msgs="Can you make a pet for my dog Mac? He's 14 years old", resp_model=Pet)))
print(repr(c.struct(msgs="Tom: my cat is juma and he's 16 years old", resp_model=Pet)))

Pet(name='Mac', age=14, owner='NA', type='dog')
Pet(name='juma', age=16, owner='Tom', type='cat')


### Chat & Unions

We can go way deeper, for example this one I pulled from [pydantic docs](https://docs.pydantic.dev/latest/concepts/unions/#discriminated-unions) has a list of discriminated unions, shown by `pet_type`. For each object the model is required to return different things.

You should be able to use the full power of Pydantic here. I've found that instructor for Claude fails on this example.

Each sub BaseModel may also have docstrings describing usage. I've found prompting this way to be quite reliable.

In [54]:
class Cat(BaseModel):
    pet_type: Literal['cat']
    meows: int


class Dog(BaseModel):
    pet_type: Literal['dog']
    barks: float


class Reptile(BaseModel):
    pet_type: Literal['lizard', 'dragon']
    scales: bool

# Dummy to show doc strings
class Create(BaseModel):
    "Pass as final member of the `pet` list to indicate success"
    pet_type: Literal['create']

class OwnersPets(BaseModel):
    """
    Information for to gather for an Owner's pets
    """
    pet: List[Union[Cat, Dog, Reptile, Create]] = Field(..., discriminator='pet_type')

chat = Chat(model)
pr = "hello I am a new owner and I would like to add some pets for me. I have a dog which has 6 barks, a dragon with no scales, and a cat with 2 meows"
chat.struct(OwnersPets, pr=pr)

OwnersPets(pet=[Dog(pet_type='dog', barks=6.0), Reptile(pet_type='dragon', scales=False), Cat(pet_type='cat', meows=2), Create(pet_type='create')])

In [49]:
chat.struct(OwnersPets, pr="actually my dragon does have scales, can you change that for me?")

OwnersPets(pet=[Dog(pet_type='dog', barks=6.0), Reptile(pet_type='dragon', scales=True), Cat(pet_type='cat', meows=2)])

While the struct uses tool use to enforce the schema, we save in history as the `repr` response to keep the user,assistant,user flow.

In [50]:
chat.h

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'hello I am a new owner and I would like to add some pets for me. I have a dog which has 6 barks, a dragon with no scales, and a cat with 2 meows'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': "OwnersPets(pet=[Dog(pet_type='dog', barks=6.0), Reptile(pet_type='dragon', scales=False), Cat(pet_type='cat', meows=2)])"}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'actually my dragon does have scales, can you change that for me?'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': "OwnersPets(pet=[Dog(pet_type='dog', barks=6.0), Reptile(pet_type='dragon', scales=True), Cat(pet_type='cat', meows=2)])"}]}]

Alternatively you can use struct as tool use flow with `treat_as_output=False` (but requires the next input to be assistant)

In [51]:
chat.struct(OwnersPets, pr=pr, treat_as_output=False)
chat.h[-3:]

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'hello I am a new owner and I would like to add some pets for me. I have a dog which has 6 barks, a dragon with no scales, and a cat with 2 meows'}]},
 {'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01QwvZBqPf5kwQX6vGUHD7tE', input={'pet': [{'pet_type': 'dog', 'barks': 6}, {'pet_type': 'dragon', 'scales': False}, {'pet_type': 'cat', 'meows': 2}]}, name='OwnersPets', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01QwvZBqPf5kwQX6vGUHD7tE',
    'content': "pet=[Dog(pet_type='dog', barks=6.0), Reptile(pet_type='dragon', scales=False), Cat(pet_type='cat', meows=2)]"}]}]

(So I couldn't prompt it again here, next input would have to be an assistant)

### User Creation & few-shot examples

You can even add few shot examples _for each input_

In [52]:
class User(BaseModel):
    "User creation tool"
    age: int = Field(description='Age of the user')
    name: str = Field(title='Username')
    password: str = Field(
        json_schema_extra={
            'title': 'Password',
            'description': 'Password of the user',
            'examples': ['Monkey!123'],
        }
    )
res = c.struct(msgs=["Can you create me a new user for tom age 22"], resp_model=User, sp="for a given user, generate a similar password based on examples")
print(res)

age=22 name='tom' password='Monkey!123'


Uses the few-shot example as asked for in the system prompt.

## Signature:

```python
Client.struct(
    self: claudette.core.Client,
    msgs: list,
    resp_model: type[BaseModel], # non-initialized Pydantic BaseModel
    **, # Client.__call__ kwargs...
) -> BaseModel

```

```python
Chat.struct(
    self: claudette.core.Chat,
    resp_model: type[BaseModel], # non-initialized Pydantic BaseModel
    treat_as_output=True, # In chat history, tool is reflected
    **, # Chat.__call__ kwargs...
) -> BaseModel
```


In [82]:
from pydantic import BaseModel, Field
from typing import Literal, List

class Fish(BaseModel):
    """
    Represent a fish for sale. When suggesting a fish:
    - Highlight its unique features and care requirements
    - Mention its personality traits
    - Suggest a name from the examples, as if you're naming it for the customer
    """
    resp_type: Literal['fish']
    species: str
    name: str = Field(
        description="Suggested name for the fish",
        json_schema_extra={
            "examples": ["Bubbles", "Finn", "Nemo", "Splash", "Goldie"]
        }
    )
    tank_size: float = Field(description="Recommended tank size in gallons")
    temperament: str = Field(description="Fish's general behavior and compatibility")
    lifespan: str = Field(description="Expected lifespan of the fish")

class CustomerResponse(BaseModel):
    """
    Represent the customer's response. When speaking as the customer:
    - Express interest or concerns about the suggested fish
    - Ask questions about care, feeding, or compatibility
    - Show enthusiasm or hesitation based on the information provided
    """
    resp_type: Literal['customer']
    response: str

class SalespersonResponse(BaseModel):
    """
    Represent the salesperson's response. When speaking as the salesperson:
    - Address the customer's questions or concerns
    - Provide additional information about fish care
    - Suggest accessories or additional fish that might be compatible
    - Use a friendly and informative tone to build trust
    """
    resp_type: Literal['sales']
    response: str

class FishStoreConversation(BaseModel):
    """
    Simulate a conversation between a customer and a salesperson in a fish store.
    The conversation should flow naturally, with the salesperson suggesting fish
    and the customer responding. Use the Fish, CustomerResponse, and SalespersonResponse
    models alternately to create a realistic dialogue.
    """
    conversation: List[Union[Fish, CustomerResponse, SalespersonResponse]] = Field(..., discriminator='resp_type')

In [83]:
from claudette.core import *
import claudette_pydantic

model = models[-1]  # Use the latest model
chat = Chat(model)

# Start the conversation
initial_prompt = "I'm interested in buying a fish for my new apartment. Can you help me choose one?"

result = chat.struct(
    FishStoreConversation,
    pr=initial_prompt,
    sp="Simulate a conversation between a fish store salesperson and a customer. The salesperson should suggest a fish, and the conversation should continue for a few exchanges. Use the docstrings to guide the conversation flow and content."
)

print(repr(result))

FishStoreConversation(conversation=[Fish(resp_type='fish', species='Betta Fish', name='Bubbles', tank_size=5.0, temperament='Peaceful but territorial, does well in a small tank on its own', lifespan='2-5 years'), CustomerResponse(resp_type='customer', response="Ooh, a Betta fish sounds interesting! I've heard they have beautiful fins. How much space do they need and what kind of care do they require?"), SalespersonResponse(resp_type='sales', response="Betta fish are a great choice for a small apartment! They only need a 5 gallon tank, which is perfect for your space. They're relatively low maintenance - just need their water changed a couple times a week and some fish flakes to eat. Their long, flowing fins make them very eye-catching. And they have such fun personalities, they can be quite interactive with their owners. What do you think, does a Betta sound like a good fit?"), CustomerResponse(resp_type='customer', response='That sounds perfect! I love the idea of having a little fish

In [96]:
print(result.json())

{"conversation":[{"resp_type":"fish","species":"Betta Fish","name":"Bubbles","tank_size":5.0,"temperament":"Peaceful but territorial, does well in a small tank on its own","lifespan":"2-5 years"},{"resp_type":"customer","response":"Ooh, a Betta fish sounds interesting! I've heard they have beautiful fins. How much space do they need and what kind of care do they require?"},{"resp_type":"sales","response":"Betta fish are a great choice for a small apartment! They only need a 5 gallon tank, which is perfect for your space. They're relatively low maintenance - just need their water changed a couple times a week and some fish flakes to eat. Their long, flowing fins make them very eye-catching. And they have such fun personalities, they can be quite interactive with their owners. What do you think, does a Betta sound like a good fit?"},{"resp_type":"customer","response":"That sounds perfect! I love the idea of having a little fish friend to keep me company. A 5 gallon tank is manageable for

In [99]:
import json
print(json.dumps(json.loads(result.json()), indent=2))

{
  "conversation": [
    {
      "resp_type": "fish",
      "species": "Betta Fish",
      "name": "Bubbles",
      "tank_size": 5.0,
      "temperament": "Peaceful but territorial, does well in a small tank on its own",
      "lifespan": "2-5 years"
    },
    {
      "resp_type": "customer",
      "response": "Ooh, a Betta fish sounds interesting! I've heard they have beautiful fins. How much space do they need and what kind of care do they require?"
    },
    {
      "resp_type": "sales",
      "response": "Betta fish are a great choice for a small apartment! They only need a 5 gallon tank, which is perfect for your space. They're relatively low maintenance - just need their water changed a couple times a week and some fish flakes to eat. Their long, flowing fins make them very eye-catching. And they have such fun personalities, they can be quite interactive with their owners. What do you think, does a Betta sound like a good fit?"
    },
    {
      "resp_type": "customer",
     